In [1]:
import torch

In [4]:
target = torch.randn(3, 4, 5)
output = torch.randn(3, 4, 5)
t = torch.randn(3)
beta_1 = torch.randn(3)

In [3]:
batch_size, seq_len, K = target.shape

In [7]:
torch.mean(torch.sum((target - output) ** 2, dim=(-2, -1)) * K * beta_1 * t / seq_len)

tensor(19.7453)

In [8]:
target = target.repeat(2, 3, 1)
output = output.repeat(2, 3, 1)
t = t.repeat(2)
beta_1 = beta_1.repeat(2)

In [10]:
batch_size, seq_len, K = target.shape

In [11]:
torch.mean(torch.sum((target - output) ** 2, dim=(-2, -1)) * K * beta_1 * t / seq_len)

tensor(19.7453)